# Getting Start

## Install JanusQ

JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it. Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker (Recommended)**

Pull docker image

```bash
  docker pull janusq/janusq:latest
```

Run docker image

```bash
  docker run -itd -p 8888:22 -p 9999:23 --name tutorial janusq/janusq
```

The docker can be accessed via

```bash
  ssh root@localhost -p 8888
```

or

```bash
  docker exec -it tutorial bash
```

The source code is in "/JanusQ/janusq" and examples can be directly run in "/JanusQ/examples/ipynb". The jupyter notebook can be visited in http://localhost:9999/lab.

**From wheel**

Download [janusq.whl](https://github.com/JanusQ/JanusQ/blob/main/dist/janusq-0.1.0-py3-none-any.whl) and install with `pip`

```bash
  pip install janusq.whl
```  

**From source code**

Pull the source code from github and install the dependencies

```bash
  git clone git@github.com:JanusQ/JanusQ.git
  cd JanusQ
  conda create -n janusq python=3.10
  pip install -r requirements.txt
```  

Set up for HyQSAT

```bash
  cd ./janusq/application/hyqsat/solver
  cmake .
  make install
```  

Set up for Choco-Q

- For Linux with CPU

```bash
  cd ./janusq/application/chocoq
  conda env create -f environment_linux_cpu.yml
```

- For Linux with GPU

```bash
  cd ./janusq/application/chocoq
  conda env create -f environment_linux_gpu.yml
```

- For MasOS

```bash
  cd ./janusq/application/chocoq
  conda env create -f environment_macos.yml
```

## Structure of JanusQ

- QuCT
  - Vectorization
    - "janusq/analysis/vectorization.py": Python script to generate the path table and vecterization of circuits, which serves as a upstream model for various analysis and optimization tasks.
  - Fidelity preidiction and optimization
    - "janusq/analysis/fidelity_prediction.py": Python script to train a downstream model for circuit fidelity prediction.
    - "janusq/optimization/mapping/mapping_ct.py"：Python script of a typical compilation flow, including routing and scheduling. The compilation flow transforms the circuit to satisfy the processor topology so as to optimize the circuit fidelity based on the prediction model.
  - Unitary decomposition
    - "janusq/analysis/unitary_decompostion.py": Python script that takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.
- MorphQPV
  - "janusq/verification/morphqpv": Directory holding the implementation of the MorphQPV algorithm, which is a quantum program verification tool.
- QuFEM
  - "janusq/calibration/readout_mitigation/qufem/benchmarking.py": Python script to generate a circuit for measuring calibration matrices.
  - "janusq/calibration/readout_mitigation/qufem/mitigation.py": Python script to implement iterative calibration, which includes quantifying interactions between qubits and constructing Bayesian networks.
  - "janusq/calibration/readout_mitigation/qufem/tools.py": Python script to implement data format conversion.
- HyQSAT
  - "janusq/application/hyqsat/common": Directory holding common functions about cnf files, such as "read_cnf.py".
  - "janusq/application/hyqsat/solver/solver.py": Python script holding different APIs used to solve sat problem.
- Choco-Q
  - "janusq/application/chocoq": Directory holding the implementation of Choco-Q.

## Q&A

1. Error "Permission denied 'minisat_core'"

    Run following command

    ```bash
      chmod +x janusq/application/hyqsat/solver/minisat_core
    ```

2. Error "Library not loaded: @rpath/libpython3.10.dylib" 

    Run following command

    ```bash
      install_name_tool -add_rpath /path/to/python3.10/lib janusq/application/hyqsat/solver/minisat_core
    ```

## Test JanusQ

In [1]:
import os
os.chdir("../..")
import logging
logging.basicConfig(level=logging.WARN)

### Test QuCT

In [2]:
# test QuCT
from janusq.objects.backend import LinearBackend
from janusq.objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel

# define backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

# construct circuit dataset
circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

# train vectorization model
up_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
up_model.train(circuits, multi_process=False, remove_redundancy = False)

100%|██████████| 300/300 [00:02<00:00, 141.79it/s]


### Test QuFEM

In [3]:
# test QuFEM (readout calibration)
from janusq.objects.backend import LinearBackend
from janusq.objects.algorithms import get_algorithm_circuits
from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.calibration.readout_mitigation.qufem import EnumeratedProtocol

# define backend and circuit
n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

# simulate noisy readout
samples = 10000
simulator = NoisySimulator(backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)

# readout calibration
protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()
all_statuscnts = [
    simulator.execute(cir_protocol, samples) 
    for cir_protocol in circuits_protocol
]
print(all_statuscnts)

/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


[{'111': 1, '011': 12, '101': 8, '110': 56, '001': 125, '100': 722, '010': 604, '000': 8472}, {'110': 4, '100': 42, '001': 8065, '011': 597, '111': 55, '101': 639, '010': 43, '000': 555}, {'101': 9, '011': 8, '100': 715, '001': 117, '110': 44, '010': 592, '000': 8515}, {'001': 3, '110': 781, '100': 11, '111': 7, '011': 121, '000': 184, '010': 8893}, {'101': 11, '110': 43, '001': 151, '011': 8482, '111': 690, '000': 17, '010': 606}, {'111': 9, '011': 135, '010': 8923, '000': 155, '100': 22, '110': 756}, {'101': 10, '011': 8, '001': 120, '100': 689, '110': 50, '010': 610, '000': 8513}, {'000': 567, '010': 37, '111': 53, '011': 566, '101': 697, '110': 7, '100': 47, '001': 8026}, {'101': 10, '111': 1, '011': 11, '001': 118, '100': 730, '110': 52, '010': 639, '000': 8439}, {'111': 7, '000': 65, '010': 5, '101': 125, '110': 668, '100': 9130}, {'000': 6, '110': 35, '100': 576, '001': 85, '011': 6, '111': 646, '101': 8646}, {'111': 10, '010': 4, '000': 92, '101': 132, '110': 641, '100': 9121},

### Test HyQSAT

In [4]:
# test HyQSAT
from janusq.application.hyqsat import readCNF

# input cnf flie
file_path = "./examples/data/cnf_examples/UF50/uf50-01.cnf"
print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -

### Test Choco-Q

Activate the corresponding virtual environment for Choco-Q

```bash
  conda activate chocoq
```

In [1]:
import os
os.chdir("../..")
import logging
logging.basicConfig(level=logging.WARN)

# test Choco-Q
from janusq.application.chocoq.chocoq.model import LinearConstrainedBinaryOptimization as LcboModel

m = LcboModel()
x = m.addVars(5, name="x")
m.setObjective((x[0] + x[1])* x[3] + x[2], "max")
m.addConstr(x[0] + x[1] - x[2] == 0)
m.addConstr(x[2] + x[3] - x[4] == 1)
optimize = m.optimize()
print(f"Optimized cost and assignment: {optimize}\n\n")

Restricted license - for non-production use only - expires 2026-11-23
Optimized cost and assignment: (2.0, {'x_0': 0.0, 'x_1': 1.0, 'x_2': 1.0, 'x_3': 1.0, 'x_4': 1.0})


